<h4>(1 - Recommend). Create and save new notebook(s) to enable automatic re-mounting of Drive storage. Copy and paste the below code into a <a href="http://colab.research.google.com/#create=true" target="_parent">new notebook</a> in Colab and follow the directions in the <a href="../#how-to-use" target="_parent">How To Use</a> section of the <em>README</em></h4><h4>(2). Open the pre-formed version in Colab (requires manual authorization each time a notebook is opened) <a href="https://colab.research.google.com/github/tdulcet/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h4>

In [ ]:
#@title # **📔 Colab GPU and CPU Notebook**#{ vertical-output: true, display-mode: "form" }

import os

#@markdown #### ↖️ Click the ▶️ button after deciding on the options below.
#@markdown #### 🔌 Make sure the GPU is enabled under *Runtime→Change runtime type*
#@markdown #### 💡 Keep each notebook **open** in the browser to prevent disconnection. 📌 [the tab](https://support.mozilla.org/kb/pinned-tabs-keep-favorite-websites-open) or move them to a dedicated window for easy access.
#@markdown #### ℹ️ This notebook uses both our GpuOwl and Prime95 [Bash install scripts](https://github.com/tdulcet/Distributed-Computing-Scripts#organizations) and our [PrimeNet Python script](https://github.com/tdulcet/Distributed-Computing-Scripts#primenet).
#@markdown #### 📜 Please see the [documentation](https://github.com/tdulcet/Distributed-Computing-Scripts/tree/master/google-colab) for more information and to ❤️ support us.
#@markdown #### 🤷 Optionally, create a GIMPS/PrimeNet account [here](https://www.mersenne.org/update/) and [join](https://www.mersenne.org/jteam/) the “Portland State University” team!

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
GPU_type_of_work = '150 - First time PRP tests' #@param ['4 - P-1 factoring', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '153 - 100 million digit PRP tests', '154 - First time PRP tests that need P-1 factoring', '155 - Double-check tests using PRP with proof']
GPU_prp_proof_power = '5' #@param ['4', '5', '6', '7', '8', '9', '10', '11', '12']
CPU_type_of_work = '150 - First time PRP tests' #@param ['0 - Whatever makes the most sense', '1 - Trial factoring to low limits', '2 - Trial factoring', '4 - P-1 factoring', '5 - ECM for first factors of Mersenne numbers', '6 - ECM on Fermat numbers', '8 - ECM on Mersenne cofactors', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '153 - 100 million digit PRP tests', '154 - First time PRP tests that need P-1 factoring', '155 - Double-check tests using PRP with proof', '160 - First time PRP on Mersenne cofactors', '161 - Double-check PRP on Mersenne cofactors']
CPU_prp_proof_power = '5' #@param ['Default', '5', '6', '7', '8', '9', '10', '11', '12']
CPU_prp_proof_power_mult = 'Default' #@param ['Default', '1', '2', '3', '4']
#@markdown Every lower power halves Drive storage requirements for PRP tests, but doubles the certification cost. Drive storage (GiB) needed for PRP tests:

#@markdown Proof<br>Power | Exponent 50M | Exponent 100M | Exponent 150M | Exponent 200M | Exponent 250M | Exponent 300M | Exponent 332.1M<br>(100M digits)
#@markdown --- | ---: | ---: | ---: | ---: | ---: | ---: | ---:
#@markdown 5 | 0.186 | 0.372 | 0.558 | 0.745 | 0.931 | 1.117 | 1.237
#@markdown 6 | 0.372 | 0.745 | 1.117 | 1.49 | 1.862 | 2.235 | 2.475
#@markdown 7 | 0.745 | 1.49 | 2.235 | **2.98** | **3.725** | **4.47** | **4.95**
#@markdown 8 | 1.49 | **2.98** | **4.47** | 5.96 | 7.45 | 8.94 | 9.9
#@markdown 9 | ***2.98*** | *5.96* | 8.94 | 11.9 | 14.9 | 17.88 | 19.8
#@markdown 10 | 5.96 | 11.92 | *17.88* | *23.84* | *29.8* | *35.76* | *39.6*
#@markdown 11 | 11.92 | 23.84 | 35.76 | 47.68 | 59.6 | 71.52 | 79.2
#@markdown 12 | 23.84 | 47.68 | 71.52 | 95.36 | 119.2 | 143 | 158.4
CPU_proof_certification_work = True #@param {type:"boolean"}
computer_number = 'Default (1)' #@param ['Default (1)', '2', '3', '4'] {allow-input: true}
output_type = 'GPU (GpuOwl)' #@param ['GPU and CPU', 'GPU (GpuOwl)', 'CPU (Prime95)']
#@markdown #### 🦉 GpuOwl version to use for each worktype:
gpuowl_prp_pm1 = 'v7.2-112' #@param ['v7.2-112', 'v6']
gpuowl_prp = 'Latest' #@param ['Latest', 'v7.2-112', 'v6']
gpuowl_ll = 'v6' #@param ['v6']
gpuowl_pm1 = 'v6' #@param ['Latest', 'v6']
local_time = 'Pacific' #@param ['Pacific', 'Mountain', 'Central', 'Eastern', 'Alaska', 'Hawaii']
debug = 'False' #@param ['False', 'GPU (GpuOwl)', 'CPU (Prime95)']

#@markdown #### 🐛 The *debug* option outputs GPU (GpuOwl) or CPU (Prime95/MPrime) progress and status, then exits.

gpu_info = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this Notebook

class StopExecution(Exception):
  def _render_traceback_(self):
      pass

def run():
  '''Run GpuOwl and MPrime'''
  print('\nStarting PrimeNet\n')
  adir = f'gpuowl/{computer_number}'
  !cd '{adir}'; nohup python3 ../primenet.py -t 3600 -T {GPU_type_of_work} >> primenet.out &
  !sleep 1
  while not os.path.exists(f'{adir}/worktodo.ini'):
    print("Waiting for 'worktodo.ini' access...")
    !sleep 1

  os.chmod('mprime_gpu/mprime', 0o777)
  file = f'{adir}/gpuowl'
  os.chmod(file, 0o777)
  !chmod 777 gpuowl/gpuowl-*/gpuowl

  versions = {'Latest': 1, 'v7.2-112': 2, 'v6': 3}
  !sed -i '/^GPUOWL_PRP_PM1=/c\GPUOWL_PRP_PM1={versions[gpuowl_prp_pm1]}' '{file}'
  !sed -i '/^GPUOWL_PRP=/c\GPUOWL_PRP={versions[gpuowl_prp]}' '{file}'
  !sed -i '/^GPUOWL_LL=/c\GPUOWL_LL={versions[gpuowl_ll]}' '{file}'
  !sed -i '/^GPUOWL_PM1=/c\GPUOWL_PM1={versions[gpuowl_pm1]}' '{file}'
  !sed -i '/^PROOF_POWER=/c\PROOF_POWER={GPU_prp_proof_power}' '{file}'

  if output_type == 'GPU and CPU':
    print('\nStarting Prime95\n')
    !cd mprime_gpu; nohup ./mprime -A{computer_number} -d >> '{'cpu' + computer_number + '.out'}' &
    print('\nStarting GpuOwl\n')
    !cd '{adir}'; nohup ./gpuowl > gpu.out &
    ![[ -e '{adir + '/gpuowl.log'}' ]] && tail '{adir + '/gpuowl.log'}'
    !tail -f '{'mprime_gpu/cpu' + computer_number + '.out'}' '{adir + '/primenet.out'}' '{adir + '/gpu.out'}'
  elif output_type == 'GPU (GpuOwl)':
    print('\nStarting Prime95\n')
    !cd mprime_gpu; nohup ./mprime -A{computer_number} -d >> '{'cpu' + computer_number + '.out'}' &
    print('\nStarting GpuOwl\n')
    !cd '{adir}'; nohup ./gpuowl > gpu.out &
    ![[ -e '{adir + '/gpuowl.log'}' ]] && tail '{adir + '/gpuowl.log'}'
    !cd '{adir}' && tail -f primenet.out gpu.out
  elif output_type == 'CPU (Prime95)':
    print('\nStarting GpuOwl\n')
    !cd '{adir}'; nohup ./gpuowl > gpu.out &
    print('\nStarting Prime95\n')
    !cd mprime_gpu && ./mprime -A{computer_number} -d | tee -ia '{'cpu' + computer_number + '.out'}'

def install():
  '''Download/Install/Configure GpuOwl then Prime95'''
  !wget https://github.com/tdulcet/Distributed-Computing-Scripts/archive/master.zip -nv -O master.zip
  !unzip -o master.zip

  print("Downloading, building and setting up GpuOwl\n")
  !cp Distributed-Computing-Scripts-master/{gpuowl2.sh,primenet.py,idletime.sh,gpuowl-bench.sh,gpuowl-wrapper.sh} .
  !sed -i '/^GPU=/,/^fi/ s/^/# /' gpuowl2.sh # Do not check for an GPU
  !sed -i '/^if command -v clinfo/,/^fi/ s/^/# /' gpuowl2.sh # Do not check for clinfo
  !sed -i '/^time \.\/gpuowl-bench\.sh / s/^/echo "Skipped" # /' gpuowl2.sh # Do not start the benchmark
  !sed -i '/^nohup / s/^/echo "Skipped" # /' gpuowl2.sh # Do not start PrimeNet
  !sed -i '/^python3 / s/^/echo "Skipped" # /' gpuowl2.sh # Do not start PrimeNet
  !sed -i '/^crontab / s/^/echo "Skipped" # /' gpuowl2.sh # Do not create a cronjob
  !bash -- gpuowl2.sh {computer_number} '{prime_ID}' '{computer_name}' {GPU_type_of_work}
  file = f'gpuowl/{computer_number}/gpuowl'
  !sed -i '/^ARGS[12]=/a -log 100000' '{file}'
  !sed -i '/^ARGS3=/a -jacobi 100000' '{file}'
  !sed -i '/^ARGS1=/a -mprimeDir ../../mprime_gpu' '{file}'
  !sed -i 's/0.9/0.95/' '{file}' # Use up to 95% of GPU RAM
  print("Registering computer with PrimeNet\n")
  !cd '{'gpuowl/' + computer_number}' && python3 ../primenet.py -t 0 -W 1 -T {GPU_type_of_work} -u '{prime_ID}' -r 'results.ini' -g -H '{computer_name}'

  print("Downloading and setting up Prime95\n")
  !cp Distributed-Computing-Scripts-master/{mprime2.sh,mprime2.exp} .
  !sed -i 's/"mprime"/"mprime_gpu"/' mprime2.sh # Name the folder specific to the runtime type
  !sed -i '/^\.\/mprime / s/^/echo "Skipped" # /' mprime2.sh # Do not start Prime95
  !sed -i '/^nohup / s/^/echo "Skipped" # /' mprime2.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/echo "Skipped" # /' mprime2.sh # Do not create a cronjob
  !sed -i '/^expect {/a \\t"Max emergency memory in GB/worker (*):" { sleep 1; send -- "3\\r"; exp_continue }\n\t"Get occasional proof certification work (*):" { sleep 1; send -- "CPU_PROOF_CERTIFICATION_WORK\\r"; exp_continue }\n\t"Minutes between disk writes (*):" { sleep 1; send -- "10\\r"; exp_continue }\n\t"Days of work to queue up (*):" { sleep 1; send -- "1\\r"; exp_continue }' mprime2.exp
  !sed -i 's/CPU_PROOF_CERTIFICATION_WORK/'$CPU_proof_certification_work'/' mprime2.exp
  !sed -i 's/0.25/0.125/' mprime2.exp
  if os.path.exists('mprime_gpu'):
    os.chmod('mprime_gpu/mprime', 0o777)
  !bash -- mprime2.sh {computer_number} '{prime_ID}' '{computer_name}' {CPU_type_of_work} # Run script
  file = f'mprime_gpu/prim{int(computer_number):04d}.txt'
  !echo 'FixedHardwareUID=1' > temp.txt
  if CPU_prp_proof_power != 'Default':
    !echo 'ProofPower={CPU_prp_proof_power}' >> temp.txt
  if CPU_prp_proof_power_mult != 'Default':
    !echo 'ProofPowerMult={CPU_prp_proof_power_mult}' >> temp.txt
  !cat '{file}' >> temp.txt
  !mv temp.txt '{file}'
  file = f'mprime_gpu/loca{int(computer_number):04d}.txt'
  !echo 'PreallocateDisk=0' >> '{file}'
  run()

def debug_exit():
  '''Output GPU and output of Prime95 or GpuOwl output'''
  if debug == 'GPU (GpuOwl)' and os.path.exists(f'gpuowl/{computer_number}/gpu.out'):
    print(f"\nOutput for computer number {computer_number}:\n")
    print("\nPrimeNet output:\n")
    !tail -n 100 '{'gpuowl/' + computer_number + '/primenet.out'}' # view primenet output
    print("\nGPU (GpuOwl) output: ")
    !tail -n 100 '{'gpuowl/' + computer_number + '/gpu.out'}' # view GpuOwl progress
    !cd '{'gpuowl/' + computer_number}' && python3 ../primenet.py -s
    print()
  elif debug == 'CPU (Prime95)' and os.path.exists(f'mprime_gpu/cpu{computer_number}.out'):
    print("\nCPU (Prime95) output:\n")
    !tail -n 100 '{'mprime_gpu/cpu' + computer_number + '.out'}' # view MPrime progress
    os.chmod('mprime_gpu/mprime', 0o777)
    !cd mprime_gpu && ./mprime -s -A{computer_number}
    print()
  else:
    print(f'No `{debug!r}` output file found for debug option and computer number `{computer_number!r}`.\n')

def load_drive():
  '''Load & cd into gdrive for persistent data'''
  if os.path.exists("/content/drive/My Drive"): # create your own notebook with our code
    %cd "/content/drive/My Drive"
  else: # use our notebook
    print('Warning: Google Drive is not mounted')
    print('If you were not expecting this, on the far left click the folder icon, the "Mount Drive" folder button, select "CONNECT TO GOOGLE DRIVE" ')
    print('and then re-execute this cell.')
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd "/content/gdrive/My Drive"
  if "My Drive/GIMPS" in os.getcwd(): # don't create a subfolder in GIMPS/
    return
  os.makedirs('GIMPS', exist_ok=True)
  %cd "GIMPS"

def gpu_check():
  '''GPU Check'''
  global gpu_info
  gpu_info = "\n".join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Select the "Runtime" → "Change runtime type" → "GPU" → "SAVE" to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
    raise StopExecution
  print(f'\nGraphics Processor (GPU):\t{gpu_info}\n')

gpu_check()
!wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
!python3 -V
print()
load_drive()

# set local time
!rm -f /etc/localtime
!ln -s {'/usr/share/zoneinfo/US/' + local_time} /etc/localtime

# use/cleanup input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computer_name = "" if computer_name.lower() == 'default' else computer_name
computer_number = "1" if computer_number.lower() == 'default (1)' else computer_number.strip()
CPU_type_of_work = CPU_type_of_work.split("-", 1)[0].rstrip()
CPU_proof_certification_work = "y" if CPU_proof_certification_work else "n"
GPU_type_of_work = GPU_type_of_work.split("-", 1)[0].rstrip()
debug = False if debug == 'False' else debug


if debug:
  debug_exit()
  raise StopExecution

if not computer_number.isdigit() or int(computer_number) < 0:
  print("ERROR: Computer number must be a number")
  raise StopExecution

elif os.path.exists(f'mprime_gpu/work{int(computer_number):04d}.txt') and os.path.exists(f'gpuowl/{computer_number}/local.ini'):
  !cd mprime_gpu && echo -e "$(date)\t$(sed -n 's/^model name[[:space:]]*: *//p' /proc/cpuinfo | uniq)  $(sed -n 's/^model[[:space:]]*: *//p' /proc/cpuinfo | uniq)" >> cpus.txt
  print('\nPrevious CPU counts')
  !cd mprime_gpu; cut -f 2 cpus.txt | sort | uniq -c | sort -nr
  !cd gpuowl && echo -e "$(date)\t$gpu_info" >> gpus.txt
  print('\nPrevious GPU counts')
  !cd gpuowl; cut -f 2 gpus.txt | sort | uniq -c | sort -nr
  run()

else:
  install()

print("Gracefully exiting...")